In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [2]:
REBUILD_INPUTS = True
STATE = 'before'
N_ORIGIN_CELLS = 2500
SAMPLE_RANGE = [0, 2500]

In [3]:
# read transport network and timetable for r5
if 1:
    r5_transit_network = r5py.TransportNetwork(
        os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip'),
    )

In [4]:
if REBUILD_INPUTS:
    G_modes = {}
    L_modes = {}

In [5]:
if REBUILD_INPUTS:
    # Load small street graph
    G = snman.io.load_street_graph(
        os.path.join(data_directory, outputs_path, 'G_edges_small.gpkg'),
        os.path.join(data_directory, outputs_path, 'G_nodes_small.gpkg'),
        crs=CRS_internal
    )
    
    if STATE == 'before':
        ln_desc = KEY_LANES_DESCRIPTION
    else:
        ln_desc = KEY_LANES_DESCRIPTION_AFTER
    
    # create mode graphs for other modes
    for mode in [MODE_CYCLING, MODE_FOOT]:
        print(f'Make street and lane graphs for {mode}')
        G_modes[mode] = copy.deepcopy(G)
        snman.street_graph.filter_lanes_by_modes(
            G_modes[mode], [mode], lane_description_key=ln_desc
        )
        L_modes[mode] = snman.lane_graph.create_lane_graph(
            G_modes[mode], lanes_attribute=ln_desc
        )


Make street and lane graphs for cycling
Make street and lane graphs for foot


In [6]:
if REBUILD_INPUTS:
    # load the new car street graph created from MATSim output
    G_modes[MODE_PRIVATE_CARS] = snman.io.load_street_graph(
        os.path.join(outputs_path, f'tt_{STATE}_edges_small.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes_small.gpkg'),
        crs=CRS_internal,
        unstringify_attributes={'lanes': snman.space_allocation.space_allocation_from_string}
    )
    
    # we must avoid that the start/end point of trips are matched onto nodes that not accessible by car
    # therefore, we reduce the graph only to those links and nodes that are accessible by car
    snman.street_graph.filter_lanes_by_modes(
        G_modes[MODE_PRIVATE_CARS], [MODE_PRIVATE_CARS], lane_description_key='lanes'
    )
    
    L_modes[MODE_PRIVATE_CARS] = snman.lane_graph.create_lane_graph(
        G_modes[MODE_PRIVATE_CARS],
        lanes_attribute='lanes',
        cast_attributes={'matsim_tt_cars_7:00': 'car_7:00', 'matsim_tt_cars': 'car_18:00'}
    )
    

In [7]:
# Import Statent grid
if REBUILD_INPUTS:
    statent = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
    )
    statent.rename(columns={'RELI': 'id'}, inplace=True)

In [8]:
if REBUILD_INPUTS:
    # read statpop that has been pre-joined with statent
    statpop_with_statent_ids = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
    )

In [9]:
perimeter = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), index='id'
).geometry['ebc_zrh_v01'].simplify(500, preserve_topology=True)
statent_in_perimeter = statent[statent.within(perimeter)]

In [10]:
cells = list(statent_in_perimeter['id'].sample(n=N_ORIGIN_CELLS, random_state=0))[SAMPLE_RANGE[0]:SAMPLE_RANGE[1]]

def miniwrapper(args):
    cell, i, N = args
    print(f'calculating cell {cell} ({i}/{N})')
    a = snman.accessibility.calculate_accessibility_for_statent_cell(
        r5_transit_network,
        G_modes,
        L_modes,
        statpop_with_statent_ids, statent, cell,
        datetime.datetime(2024, 2, 22, 18, 00),
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.1
    )
    return a

accessibility = pd.concat(
    map(
        miniwrapper,
        zip(
            cells,
            range(len(cells)),
            [len(cells)] * len(cells)
        )
    ),
    ignore_index=True
)


print('exporting file')
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, f'accessibility_{STATE}.gpkg')
)

calculating cell 70042281 (0/10)
calculating cell 68132486 (1/10)
calculating cell 68672447 (2/10)
private_cars
35034.0 5640.22938428576
foot
37411.0 5372.87868535556
cycling
37411.0 5372.87868535556
calculating cell 69702513 (3/10)
private_cars
25027.0 12051.365439290348
foot
37411.0 10824.962907467203
cycling
37411.0 10824.962907467203
calculating cell 67962611 (4/10)
private_cars
40807.0 6968.425926791053
foot
27372.0 6688.439376829847
cycling
27372.0 6688.439376829847
calculating cell 66382604 (5/10)
calculating cell 68322416 (6/10)
calculating cell 66682596 (7/10)
private_cars
49523.0 15044.987801670113
foot
26936.0 14424.346966994868
cycling
26936.0 14424.346966994868
calculating cell 68172475 (8/10)
private_cars
35034.0 1893.233362278431
foot
28723.0 2435.965763279687
cycling
28723.0 2435.965763279687
calculating cell 69482396 (9/10)
exporting file


In [11]:
accessibility

,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility,accessibility_cycling,accessibility_foot,accessibility_private_cars,accessibility_transit,geometry
0,7750768,38,1,1,301,3,68672447.0,91257.976667,57459.793185,6752.319155,99520.441153,71665.772926,POINT (2686663.001 1244743.001)
1,9811619,48,2,2,-2,-2,68672447.0,91257.976667,57459.793185,6752.319155,99520.441153,71665.772926,POINT (2686744.001 1244666.001)
2,11335928,56,1,2,201,2,68672447.0,91257.976667,57459.793185,6752.319155,99520.441153,71665.772926,POINT (2686703.001 1244729.001)
3,537284,13,1,1,-2,-2,69702513.0,74721.248913,37250.019776,1025.197553,82629.089801,52318.234065,POINT (2697032.001 1251290.001)
4,3295332,20,2,1,-2,-2,69702513.0,74721.248913,37250.019776,1025.197553,82629.089801,52318.234065,POINT (2696996.001 1251309.001)
5,7633444,37,1,2,-2,-2,69702513.0,74721.248913,37250.019776,1025.197553,82629.089801,52318.234065,POINT (2697040.001 1251272.001)
6,9736902,53,1,2,-2,-2,69702513.0,74721.248913,37250.019776,1025.197553,82629.089801,52318.234065,POINT (2696987.001 1251322.001)
7,11102993,18,2,1,-2,-2,69702513.0,74721.248913,37250.019776,1025.197553,82629.089801,52318.234065,POINT (2696970.001 1251256.001)
8,4353646,52,2,2,302,3,67962611.0,85318.161684,43443.083363,2213.705543,93904.963654,54762.962021,POINT (2679635.001 1261059.001)
9,4944943,27,1,2,201,2,67962611.0,85318.161684,43443.083363,2213.705543,93904.963654,54762.962021,POINT (2679618.001 1261094.001)
